In [ ]:
!pip install requests
!pip install BeautifulSoup
!pip install langchain

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip uninstall spacy cymem murmurhash preshed thinc blis -y
!pip install spacy
!pip install -numpy
#!spacy.prefer_gpu()
!pip install scispacy
!pip install pdfplumber
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
# !pip install pymupdf transformers torch torchvision torchaudio
!pip install https://s3.amazonaws.com/models.huggingface.co/bert/en_core_sci_md.tar.gz

Found existing installation: spacy 3.7.5
Uninstalling spacy-3.7.5:
  Successfully uninstalled spacy-3.7.5
Found existing installation: cymem 2.0.11
Uninstalling cymem-2.0.11:
  Successfully uninstalled cymem-2.0.11
Found existing installation: murmurhash 1.0.12
Uninstalling murmurhash-1.0.12:
  Successfully uninstalled murmurhash-1.0.12
Found existing installation: preshed 3.0.9
Uninstalling preshed-3.0.9:
  Successfully uninstalled preshed-3.0.9
Found existing installation: thinc 8.2.5
Uninstalling thinc-8.2.5:
  Successfully uninstalled thinc-8.2.5
Found existing installation: blis 0.7.11
Uninstalling blis-0.7.11:
  Successfully uninstalled blis-0.7.11
  Using cached spacy-3.8.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached murmurhash-1.0.12-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached cymem-2.0.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.wh


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -n
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_comma

In [ ]:
import os
import requests
import xml.etree.ElementTree as ET
import spacy
import scispacy
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

In [ ]:
nlp = spacy.load("en_core_sci_sm")

/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_sm' (0.5.4) was trained with spaCy v3.7.4 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/spacy/language.py:2232: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able 

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

def extract_hrefs_from_url_by_title(url = 'https://arxiv.org/list/cs.AI/recent?skip=0&show=2000', target_title='View HTML'):
    try:
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        hrefs = []
        for a_tag in soup.find_all('a', title=target_title):
            if 'href' in a_tag.attrs:
                hrefs.append(a_tag['href'])
        return hrefs
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
        return []

def fetch_arxiv_ids(domains, max_results=5):
    """
    Fetches the ArXiv IDs of papers for the specified domains.

    Args:
        domains (list): A list of ArXiv subject categories (e.g., ["cs.AI", "physics.hep-th"]).
        max_results (int): The maximum number of results to fetch per domain (default: 100).

    Returns:
        list: A list of ArXiv paper IDs.
    """
    all_ids = []
    for domain in domains:
        url = f"http://export.arxiv.org/api/query?search_query=cat:{domain}&start=0&max_results={max_results}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error fetching data for domain: {domain}")
            continue

        root = ET.fromstring(response.text)
        for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
            # The ArXiv ID is typically found in the <id> tag.
            arxiv_id_full = entry.find("{http://www.w3.org/2005/Atom}id").text
            # The ID often looks like 'http://arxiv.org/abs/2304.01234v1'.
            # We want to extract just '2304.01234v1'.
            arxiv_id = arxiv_id_full.split('/')[-1]
            all_ids.append(arxiv_id)
    return all_ids


# Download PDF
def download_pdf(pdf_url, save_path="paper.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        return save_path
    return None


# Tokenization using SciSpaCy
def tokenize_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# SciBERT Embedding
def get_scibert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = scibert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import HTMLHeaderTextSplitter

def download_arxiv_html(arxiv_html_url, save_path):
    response = requests.get(arxiv_html_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    for script in soup(['script', 'style']):
        script.extract()

    cleaned_html = soup.prettify()

    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(cleaned_html)

    print(f"Saved HTML content to {save_path}")

def extract_abstract(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    abstract_div = soup.find('div', class_='ltx_abstract')
    abstract = abstract_div.get_text(strip=True) if abstract_div else ""
    if abstract_div:
        abstract_div.extract()  # remove abstract from soup
    return abstract, soup.prettify()

def split_html_by_heading(html_content):
    headers_to_split_on = [("h1", "Heading1"), ("h2", "Heading2"), ("h3", "Heading3")]
    splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    splits = splitter.split_text(html_content)

    main_paper = "\n".join(split.page_content for split in splits) if splits else ""
    return main_paper



In [ ]:
# # Example usage:
# paper_types = ['cs.AI']
# papers = extract_hrefs_from_url_by_title()
# papers = papers[1:3]
# print(papers)
# for link in papers:
#   save_as = 'paper.html'
#   download_arxiv_html(link, save_as)

#   with open(save_as, 'r', encoding='utf-8') as file:
#       html_data = file.read()

#   abstract, html_without_abstract = extract_abstract(html_data)
#   main_paper = split_html_by_heading(html_without_abstract)

#   print(abstract)
#   print("============================")
#   print("============================")
#   print("============================")
#   print("============================")
#   print("============================")
#   print("============================")
#   print("============================")
#   print("============================")
#   print("============================")
#   print("\nMain Paper:\n", main_paper[:10000])



In [ ]:
import re
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
import numpy as np
from typing import List, Optional
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

class EncoderDecoder(nn.Module):
    """
    Simple Encoder-Decoder architecture for text summarization without attention
    """
    def __init__(self,
                hidden_dim=64,
                num_layers=2,
                dropout=0.1,
                embedding_dim=128):  # SciBERT embedding dimension
        super(EncoderDecoder, self).__init__()

        # Encoder
        self.encoder = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Decoder
        self.decoder = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Output projection (to SciBERT embedding space)
        self.output_projection = nn.Linear(hidden_dim, embedding_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def encode(self, embeddings, mask=None):
        """Encode input embeddings"""
        if mask is not None:
            # Pack padded sequence
            lengths = mask.sum(1).cpu().int()
            packed_embeddings = nn.utils.rnn.pack_padded_sequence(
                embeddings,
                lengths,
                batch_first=True,
                enforce_sorted=False
            )

            # Encode
            _, (hidden, cell) = self.encoder(packed_embeddings)
        else:
            # No masking needed (all sequences same length)
            _, (hidden, cell) = self.encoder(embeddings)

        return hidden, cell

    def decode_step(self, input_embedding, hidden, cell):
        """Perform one decoding step"""
        # Decode step
        output, (hidden, cell) = self.decoder(input_embedding, (hidden, cell))

        # Project to embedding space
        pred_embedding = self.output_projection(output)

        return pred_embedding, hidden, cell


class TextSummarizationModel(nn.Module):
    """
    Complete text summarization model with BERT embeddings - without attention
    """
    def __init__(self,
                hidden_dim=64,
                num_layers=2,
                dropout=0.1,
                max_summary_length=150):
        super(TextSummarizationModel, self).__init__()

        # SciBERT model for embeddings
        # self.tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased") #Heavy model so using lighter version
        # self.scibert = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")
        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        self.bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased")
        embedding_dim = self.scibert.config.hidden_size  # 768

        # Freeze SciBERT parameters
        for param in self.scibert.parameters():
            param.requires_grad = False

        # Encoder-Decoder without attention
        self.encoder_decoder = EncoderDecoder(hidden_dim, num_layers, dropout, embedding_dim)

        # Max summary length
        self.max_summary_length = max_summary_length

    def get_scibert_embeddings(self, texts):
        """Get SciBERT embeddings for a list of texts"""
        # Tokenize
        encoded_inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
            return_attention_mask=True
        )

        # Move to device
        device = next(self.parameters()).device
        encoded_inputs = {k: v.to(device) for k, v in encoded_inputs.items()}

        # Get embeddings
        with torch.no_grad():
            outputs = self.scibert(**encoded_inputs)

        return outputs.last_hidden_state, encoded_inputs['attention_mask']

    def select_sentences(self, text: str, num_sentences: int = 5) -> str:
        """Simple sentence selection by taking first n sentences"""
        sentences = sent_tokenize(text)
        selected = sentences[:min(num_sentences, len(sentences))]
        return " ".join(selected)

    def forward(self, source_texts: List[str], target_texts: Optional[List[str]] = None, k: int = 5):
        """
        Forward pass for training or inference

        Args:
            source_texts: List of input document strings
            target_texts: List of target summary strings (for training)
            k: Number of sentences to select

        Returns:
            Predicted embeddings or text summaries
        """
        device = next(self.parameters()).device
        batch_size = len(source_texts)

        # Process each document - just take first k sentences instead of using attention
        selected_texts = [self.select_sentences(text, k) for text in source_texts]

        # Get embeddings for selected texts
        source_embeddings, source_mask = self.get_scibert_embeddings(selected_texts)

        # Encode
        hidden, cell = self.encoder_decoder.encode(source_embeddings, source_mask)

        if target_texts is not None:
            # Training mode with teacher forcing
            target_embeddings, target_mask = self.get_scibert_embeddings(target_texts)

            # Initialize outputs
            max_len = target_embeddings.size(1)
            outputs = torch.zeros_like(target_embeddings)

            # First input is first token of target
            decoder_input = target_embeddings[:, 0:1, :]

            # Teacher forcing
            for t in range(1, max_len):
                pred_embedding, hidden, cell = self.encoder_decoder.decode_step(decoder_input, hidden, cell)
                outputs[:, t-1:t, :] = pred_embedding

                # Next input is from target (teacher forcing)
                decoder_input = target_embeddings[:, t:t+1, :]

            return outputs, target_embeddings, target_mask

        else:
            # Inference mode
            # Get [CLS] token embedding as first input
            cls_embedding = self.get_scibert_embeddings(["[CLS]"])[0][:, 0:1, :]
            decoder_input = cls_embedding.repeat(batch_size, 1, 1)

            # Generate summary
            generated_embeddings = []

            for t in range(self.max_summary_length):
                pred_embedding, hidden, cell = self.encoder_decoder.decode_step(decoder_input, hidden, cell)
                generated_embeddings.append(pred_embedding)

                # Use predicted embedding as next input
                decoder_input = pred_embedding

            # Concatenate all embeddings
            all_embeddings = torch.cat(generated_embeddings, dim=1)

            # Convert embeddings to text
            summaries = self.embeddings_to_text(all_embeddings)

            return summaries

    def embeddings_to_text(self, embeddings):
        """Convert embeddings to text (approximate method)"""
        device = embeddings.device
        batch_size = embeddings.size(0)

        # Get vocabulary embeddings (simplified approach)
        vocab_size = min(10000, self.tokenizer.vocab_size)  # Limit for memory
        vocab_tokens = list(self.tokenizer.vocab.keys())[:vocab_size]

        # Get embeddings for vocabulary tokens
        vocab_inputs = self.tokenizer(vocab_tokens, return_tensors="pt", padding=True)
        vocab_inputs = {k: v.to(device) for k, v in vocab_inputs.items()}

        with torch.no_grad():
            vocab_outputs = self.scibert(**vocab_inputs)
            vocab_embeddings = vocab_outputs.last_hidden_state[:, 0, :]  # Use [CLS] token

        # Decode each sequence in the batch
        decoded_texts = []

        for b in range(batch_size):
            sequence_embeddings = embeddings[b]
            sequence_tokens = []

            for token_embedding in sequence_embeddings:
                # Find closest vocabulary token
                similarities = torch.nn.functional.cosine_similarity(
                    token_embedding.unsqueeze(0),
                    vocab_embeddings
                )
                best_idx = similarities.argmax().item()
                sequence_tokens.append(vocab_tokens[best_idx])

            # Join tokens to form text
            decoded_text = self.tokenizer.convert_tokens_to_string(sequence_tokens)
            decoded_texts.append(decoded_text)

        return decoded_texts


class TextSummarizationDataset(Dataset):
    """Dataset for text summarization"""
    def __init__(self, documents, summaries):
        assert len(documents) == len(summaries)
        self.documents = documents
        self.summaries = summaries

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, idx):
        return self.documents[idx], self.summaries[idx]


def load_data_from_strings(document_strings, summary_strings):
    """Create dataset from lists of document and summary strings"""
    return TextSummarizationDataset(document_strings, summary_strings)


def train_model(model, train_dataset, val_dataset=None, batch_size=8, num_epochs=5, lr=1e-4):
    """Train the summarization model"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda batch: ([x[0] for x in batch], [x[1] for x in batch])
    )

    if val_dataset:
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=lambda batch: ([x[0] for x in batch], [x[1] for x in batch])
        )

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Loss function
    criterion = nn.MSELoss()  # For embedding-based training

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_docs, batch_summaries in train_loader:
            # Forward pass
            pred_embeddings, target_embeddings, target_mask = model(batch_docs, batch_summaries)

            # Calculate loss on embeddings where target is not padded
            loss = criterion(
                pred_embeddings[target_mask.unsqueeze(-1).expand_as(pred_embeddings)],
                target_embeddings[target_mask.unsqueeze(-1).expand_as(target_embeddings)]
            )

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

        # Validation
        if val_dataset:
            model.eval()
            val_loss = 0

            with torch.no_grad():
                for batch_docs, batch_summaries in val_loader:
                    # Forward pass
                    pred_embeddings, target_embeddings, target_mask = model(batch_docs, batch_summaries)

                    # Calculate loss
                    loss = criterion(
                        pred_embeddings[target_mask.unsqueeze(-1).expand_as(pred_embeddings)],
                        target_embeddings[target_mask.unsqueeze(-1).expand_as(target_embeddings)]
                    )

                    val_loss += loss.item()

            avg_val_loss = val_loss / len(val_loader)
            print(f"Validation Loss: {avg_val_loss:.4f}")

    return model


def summarize_text(model, document_string, k=5):
    """Generate summary for a document string"""
    model.eval()
    with torch.no_grad():
        summary = model([document_string])[0]
    return summary


def preprocess_text(text):
    """Preprocess text to ensure proper sentence segmentation"""
    # Replace common section markers with periods if they don't have them
    text = re.sub(r'(\d+\.\d+)([A-Z])', r'\1. \2', text)  # Add period after section numbers

    # Ensure newlines are treated as sentence boundaries
    text = text.replace('\n', '. ')

    # Fix spacing issues
    text = re.sub(r'\s+', ' ', text)

    # Replace multiple periods with single ones
    text = re.sub(r'\.+', '.', text)

    return text

def select_sentences(self, text: str, num_sentences: int = 5) -> str:
    """Improved sentence selection with better tokenization"""
    # Preprocess text for better sentence segmentation
    text = preprocess_text(text)

    # Use NLTK for sentence tokenization
    try:
        sentences = sent_tokenize(text)
    except:
        # Fallback if NLTK fails
        sentences = [s.strip() + '.' for s in text.split('.') if s.strip()]

    # Filter out very short sentences (likely parsing artifacts)
    sentences = [s for s in sentences if len(s.split()) > 3]

    # Select sentences
    selected = sentences[:min(num_sentences, len(sentences))]
    return " ".join(selected)

def train_incrementally(model, documents, summaries, batch_size=1, num_epochs=1, lr=1e-4):
    """Train the model incrementally on one document at a time"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Loss function
    criterion = nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0

        for i in range(len(documents)):
            model.train()

            # Create single-item dataset
            single_doc = [documents[i]]
            single_summary = [summaries[i]]

            # Forward pass
            pred_embeddings, target_embeddings, target_mask = model(single_doc, single_summary)

            # Calculate loss on embeddings where target is not padded
            loss = criterion(
                pred_embeddings[target_mask.unsqueeze(-1).expand_as(pred_embeddings)],
                target_embeddings[target_mask.unsqueeze(-1).expand_as(target_embeddings)]
            )

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            print(f"Epoch {epoch+1}/{num_epochs}, Document {i+1}/{len(documents)}, Loss: {loss.item():.4f}")

            # Clear cache between documents
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    return model


# if __name__ == "__main__":
#        # Example usage

#     # Create model
#     model = TextSummarizationModel()

#     # Example training data
#     documents = [
#     ]

#     summaries = [

#     ]

#       # Example usage:
#     paper_types = ['cs.AI']
#     papers = extract_hrefs_from_url_by_title()
#     papers = papers[1:3]
#     print(papers)
#     for link in papers:
#       save_as = 'paper.html'
#       download_arxiv_html(link, save_as)

#       with open(save_as, 'r', encoding='utf-8') as file:
#           html_data = file.read()

#       abstract, html_without_abstract = extract_abstract(html_data)
#       main_paper = split_html_by_heading(html_without_abstract)
#       documents.append(main_paper[:10000])
#       summaries.append(abstract)

#       # print(abstract)
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("============================")
#       # print("\nMain Paper:\n", main_paper[:10000])


#     # Create dataset
#     dataset = load_data_from_strings(documents, summaries)

#     # Train model (in practice, you would need much more data)
#     # model = train_incrementally(model,documents,summaries)
#     # model = train_model(model, dataset, num_epochs=1)

#     # Generate summary
#     summary = summarize_text(model, documents)
#     print(f"Generated summary: {summary}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import re
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, DistilBertModel, DistilBertTokenizer
import numpy as np
from typing import List, Optional
import nltk
nltk.download('punkt', quiet=True)
from nltk.tokenize import sent_tokenize

class EncoderDecoder(nn.Module):
    """
    Memory-efficient Encoder-Decoder architecture for text summarization
    """
    def __init__(self,
                hidden_dim=32,  # Reduced from 64
                num_layers=1,
                dropout=0.1,
                embedding_dim=64):  # Will use dimensionality reduction
        super(EncoderDecoder, self).__init__()

        # Dimensionality reduction layer (from BERT's 768 to smaller dim)
        self.dim_reduction = nn.Linear(768, embedding_dim)

        # Encoder
        self.encoder = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0 if num_layers == 1 else dropout
        )

        # Decoder
        self.decoder = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0 if num_layers == 1 else dropout
        )

        # Output projection
        self.output_projection = nn.Linear(hidden_dim, embedding_dim)

        # Projection back to BERT space (for similarity comparison)
        self.dim_expansion = nn.Linear(embedding_dim, 768)

        # Dropout
        self.dropout = nn.Dropout(dropout)

# In the encode method of EncoderDecoder class:
    def encode(self, embeddings, mask=None):
        """Encode input embeddings"""
        # Apply dimensionality reduction
        reduced_embeddings = self.dropout(self.dim_reduction(embeddings))

        if mask is not None:
            # Pack padded sequence
            lengths = mask.sum(1).long()  # Convert to long tensor explicitly

            # Handle zero lengths
            lengths = torch.clamp(lengths, min=1)

            packed_embeddings = nn.utils.rnn.pack_padded_sequence(
                reduced_embeddings,
                lengths,
                batch_first=True,
                enforce_sorted=False
            )

            # Encode
            _, (hidden, cell) = self.encoder(packed_embeddings)
        else:
            # No masking needed (all sequences same length)
            _, (hidden, cell) = self.encoder(reduced_embeddings)

        return hidden, cell

    def decode_step(self, input_embedding, hidden, cell):
        """Perform one decoding step"""
        # Apply dimensionality reduction
        reduced_embedding = self.dim_reduction(input_embedding)

        # Decode step
        output, (hidden, cell) = self.decoder(reduced_embedding, (hidden, cell))

        # Project to embedding space
        pred_embedding = self.output_projection(output)

        # Project back to original BERT space
        expanded_embedding = self.dim_expansion(pred_embedding)

        return expanded_embedding, hidden, cell


class TextSummarizationModel(nn.Module):
    """
    Memory-efficient text summarization model
    """
    def __init__(self,
                hidden_dim=32,
                num_layers=1,
                dropout=0.1,
                max_summary_length=50,  # Reduced from 150
                reduced_dim=64):
        super(TextSummarizationModel, self).__init__()

        # Use lighter DistilBERT instead of SciBERT

        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        self.bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased")

        embedding_dim = self.bert_model.config.hidden_size  # 768

        # Freeze BERT parameters
        for param in self.bert_model.parameters():
            param.requires_grad = False

        # Encoder-Decoder
        self.encoder_decoder = EncoderDecoder(hidden_dim, num_layers, dropout, reduced_dim)

        # Max summary length
        self.max_summary_length = max_summary_length

        # Store the reduced dimension
        self.reduced_dim = reduced_dim

        # Maximum text chunk size (in tokens)
        self.max_chunk_size = 64  # Very small for memory efficiency

    def preprocess_text(self, text):
        """Preprocess text to ensure proper sentence segmentation"""
        # Replace common section markers with periods if they don't have them
        text = re.sub(r'(\d+\.\d+)([A-Z])', r'\1. \2', text)

        # Ensure newlines are treated as sentence boundaries
        text = text.replace('\n', '. ')

        # Fix spacing issues
        text = re.sub(r'\s+', ' ', text)

        # Replace multiple periods with single ones
        text = re.sub(r'\.+', '.', text)

        return text

    def select_sentences(self, text: str, num_sentences: int = 5) -> str:
        """Improved sentence selection with better tokenization"""
        # Preprocess text for better sentence segmentation
        text = self.preprocess_text(text)

        # Use NLTK for sentence tokenization
        try:
            sentences = sent_tokenize(text)
        except:
            # Fallback if NLTK fails
            sentences = [s.strip() + '.' for s in text.split('.') if s.strip()]

        # Filter out very short sentences (likely parsing artifacts)
        sentences = [s for s in sentences if len(s.split()) > 3]

        # Select sentences
        selected = sentences[:min(num_sentences, len(sentences))]
        return " ".join(selected)

    def get_bert_embeddings(self, texts, chunk_size=None):
        """Get BERT embeddings for a list of texts in small chunks"""
        if chunk_size is None:
            chunk_size = self.max_chunk_size

        device = next(self.parameters()).device

        # Process texts in small chunks to save memory
        all_embeddings = []
        all_masks = []

        for text in texts:
            # Tokenize the text
            tokens = self.tokenizer.tokenize(text)

            # Process in chunks
            chunk_embeddings = []

            for i in range(0, len(tokens), chunk_size):
                chunk = tokens[i:i + chunk_size]
                chunk_text = self.tokenizer.convert_tokens_to_string(chunk)

                # Tokenize the chunk
                encoded_inputs = self.tokenizer(
                    chunk_text,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=chunk_size,
                    return_attention_mask=True
                )

                # Move to device
                encoded_inputs = {k: v.to(device) for k, v in encoded_inputs.items()}

                # Get embeddings
                with torch.no_grad():
                    outputs = self.bert_model(**encoded_inputs)

                # Get [CLS] embedding only to save memory
                cls_embedding = outputs.last_hidden_state[:, 0, :].unsqueeze(1)
                chunk_embeddings.append(cls_embedding)

                # Clear cache
                del outputs, encoded_inputs
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            # Concatenate chunk embeddings
            if chunk_embeddings:
                text_embeddings = torch.cat(chunk_embeddings, dim=1)
                mask = torch.ones(1, text_embeddings.size(1), device=device)

                all_embeddings.append(text_embeddings)
                all_masks.append(mask)

        # Pad to same length
        max_len = max(emb.size(1) for emb in all_embeddings)
        padded_embeddings = []
        padded_masks = []

        for emb, mask in zip(all_embeddings, all_masks):
            pad_len = max_len - emb.size(1)
            if pad_len > 0:
                # Pad embeddings
                padding = torch.zeros(1, pad_len, emb.size(2), device=device)
                padded_emb = torch.cat([emb, padding], dim=1)

                # Pad mask
                mask_padding = torch.zeros(1, pad_len, device=device)
                padded_mask = torch.cat([mask, mask_padding], dim=1)

                padded_embeddings.append(padded_emb)
                padded_masks.append(padded_mask)
            else:
                padded_embeddings.append(emb)
                padded_masks.append(mask)

        # Concatenate into batch
        batch_embeddings = torch.cat(padded_embeddings, dim=0)
        batch_masks = torch.cat(padded_masks, dim=0)

        return batch_embeddings, batch_masks

    def forward(self, source_texts: List[str], target_texts: Optional[List[str]] = None, k: int = 3):
        """Forward pass with memory efficiency improvements"""
        device = next(self.parameters()).device
        batch_size = len(source_texts)

        # Process each document - just take first k sentences
        selected_texts = [self.select_sentences(text, k) for text in source_texts]

        # Get embeddings for selected texts
        source_embeddings, source_mask = self.get_bert_embeddings(selected_texts)

        # Encode
        hidden, cell = self.encoder_decoder.encode(source_embeddings, source_mask)

        if target_texts is not None:
            # Training mode with teacher forcing
            target_embeddings, target_mask = self.get_bert_embeddings(target_texts)

            # Initialize outputs
            max_len = target_embeddings.size(1)
            outputs = torch.zeros_like(target_embeddings)

            # First input is first token of target
            decoder_input = target_embeddings[:, 0:1, :]

            # Teacher forcing
            for t in range(1, max_len):
                pred_embedding, hidden, cell = self.encoder_decoder.decode_step(decoder_input, hidden, cell)
                outputs[:, t-1:t, :] = pred_embedding

                # Next input is from target (teacher forcing)
                decoder_input = target_embeddings[:, t:t+1, :]

                # Clear cache periodically
                if t % 10 == 0 and torch.cuda.is_available():
                    torch.cuda.empty_cache()

            return outputs, target_embeddings, target_mask

        else:
            # Inference mode - generate short sequences
            max_len = min(self.max_summary_length, 30)  # Limit for memory

            # Get [CLS] token embedding as first input
            cls_embedding = self.get_bert_embeddings(["[CLS]"])[0][:, 0:1, :]
            decoder_input = cls_embedding.repeat(batch_size, 1, 1)

            # Generate summary
            generated_embeddings = []

            for t in range(max_len):
                pred_embedding, hidden, cell = self.encoder_decoder.decode_step(decoder_input, hidden, cell)
                generated_embeddings.append(pred_embedding)

                # Use predicted embedding as next input
                decoder_input = pred_embedding

                # Clear cache periodically
                if t % 10 == 0 and torch.cuda.is_available():
                    torch.cuda.empty_cache()

            # Concatenate all embeddings
            all_embeddings = torch.cat(generated_embeddings, dim=1)

            # Convert embeddings to text
            summaries = self.embeddings_to_text(all_embeddings)

            return summaries

    def embeddings_to_text(self, embeddings):
        """Convert embeddings to text with memory efficiency improvements"""
        device = embeddings.device
        batch_size = embeddings.size(0)

        # Limit vocabulary size drastically for memory
        vocab_size = min(1000, self.tokenizer.vocab_size)
        vocab_tokens = list(self.tokenizer.vocab.keys())[:vocab_size]

        # Process vocabulary in small batches
        vocab_batch_size = 100
        decoded_texts = []

        for b in range(batch_size):
            sequence_embeddings = embeddings[b]
            sequence_tokens = []

            for token_embedding in sequence_embeddings:
                best_token = None
                best_similarity = -float('inf')

                # Process vocabulary in small batches
                for i in range(0, len(vocab_tokens), vocab_batch_size):
                    batch_tokens = vocab_tokens[i:i+vocab_batch_size]

                    # Get embeddings for vocabulary tokens
                    vocab_inputs = self.tokenizer(batch_tokens, return_tensors="pt", padding=True)
                    vocab_inputs = {k: v.to(device) for k, v in vocab_inputs.items()}

                    with torch.no_grad():
                        vocab_outputs = self.bert_model(**vocab_inputs)
                        vocab_embeddings = vocab_outputs.last_hidden_state[:, 0, :]

                    # Find similarities
                    similarities = torch.nn.functional.cosine_similarity(
                        token_embedding.unsqueeze(0),
                        vocab_embeddings
                    )

                    # Update best token
                    batch_best_idx = similarities.argmax().long().item()
                    batch_best_similarity = similarities[batch_best_idx].item()

                    if batch_best_similarity > best_similarity:
                        best_similarity = batch_best_similarity
                        best_token = batch_tokens[batch_best_idx]

                    # Free memory
                    del vocab_outputs, vocab_embeddings, similarities, vocab_inputs
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()

                sequence_tokens.append(best_token)

            # Join tokens to form text
            decoded_text = self.tokenizer.convert_tokens_to_string(sequence_tokens)
            decoded_texts.append(decoded_text)

            # Free memory
            del sequence_embeddings, sequence_tokens
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return decoded_texts


class TextSummarizationDataset(Dataset):
    """Dataset for text summarization"""
    def __init__(self, documents, summaries):
        assert len(documents) == len(summaries)
        self.documents = documents
        self.summaries = summaries

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, idx):
        return self.documents[idx], self.summaries[idx]


def load_data_from_strings(document_strings, summary_strings):
    """Create dataset from lists of document and summary strings"""
    return TextSummarizationDataset(document_strings, summary_strings)


def train_incrementally(model, documents, summaries, num_epochs=1, lr=1e-4, accumulation_steps=4):
    """Train the model incrementally with gradient accumulation"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Loss function
    criterion = nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0
        optimizer.zero_grad()  # Zero gradients at the beginning

        for i in range(len(documents)):
            model.train()

            # Take a small chunk of the document
            doc_chunk = documents[i][:2000]  # Limit document length

            # Create single-item dataset
            single_doc = [doc_chunk]
            single_summary = [summaries[i][:500]]  # Limit summary length

            try:
                # Forward pass
                pred_embeddings, target_embeddings, target_mask = model(single_doc, single_summary)

                # Calculate loss on embeddings where target is not padded
                loss = criterion(
                    pred_embeddings[target_mask.unsqueeze(-1).expand_as(pred_embeddings)],
                    target_embeddings[target_mask.unsqueeze(-1).expand_as(target_embeddings)]
                )

                # Scale loss for gradient accumulation
                loss = loss / accumulation_steps

                # Backward pass
                loss.backward()

                # Update only after accumulation_steps or at the end of dataset
                if (i + 1) % accumulation_steps == 0 or (i + 1) == len(documents):
                    optimizer.step()
                    optimizer.zero_grad()

                total_loss += loss.item() * accumulation_steps
                print(f"Epoch {epoch+1}/{num_epochs}, Document {i+1}/{len(documents)}, Loss: {loss.item()*accumulation_steps:.4f}")

            except RuntimeError as e:
                print(f"Error processing document {i}: {e}")
                # Skip this document and continue
                optimizer.zero_grad()
                continue

            # Clear cache between documents
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Force garbage collection
            gc.collect()

    return model


def summarize_text(model, document_string, k=3):
    """Generate summary for a document string"""
    model.eval()

    # Take only first 2000 chars to save memory
    document_string = document_string[:2000]

    with torch.no_grad():
        try:
            summary = model([document_string])[0]

            # Clean up the summary
            summary = re.sub(r'\s+', ' ', summary)
            summary = re.sub(r'[^\w\s\.,;:!?]', '', summary)

            return summary
        except Exception as e:
            print(f"Error generating summary: {e}")
            return "Error generating summary. The document may be too large."


# Example usage
if __name__ == "__main__":
    # Create model with reduced dimensions
    model = TextSummarizationModel(
        hidden_dim=32,          # Reduced from original
        num_layers=1,           # Single layer
        dropout=0.1,
        max_summary_length=30,  # Shorter summaries
        reduced_dim=64          # Lower dimensional internal representation
    )

    # Initialize empty lists
    documents = []
    summaries = []

    # Example usage:
    paper_types = ['cs.AI']
    papers = extract_hrefs_from_url_by_title()  # Your function
    papers = papers[1:2]  # Just one paper at a time

    for link in papers:
        # Download and process one paper
        save_as = 'paper.html'
        download_arxiv_html(link, save_as)  # Your function

        with open(save_as, 'r', encoding='utf-8') as file:
            html_data = file.read()

        abstract, html_without_abstract = extract_abstract(html_data)  # Your function
        main_paper = split_html_by_heading(html_without_abstract)  # Your function

        # Process just this one paper - take only first 2000 chars
        paper_chunk = main_paper[:2000]

        print(f"Processing paper: {link}")

        try:
            print("In try bkock")
            # Train incrementally on this one paper
            model = train_incrementally(
                model,
                [paper_chunk],  # Just one document
                [abstract],     # Just one summary
                num_epochs=1,
                accumulation_steps=4  # Gradient accumulation
            )

            # Clear memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Test the model on this paper
            summary = summarize_text(model, paper_chunk)
            print(f"Paper: {link}")
            print(f"Abstract: {abstract}")
            print(f"Generated summary: {summary}")

        except Exception as e:
            print(f"Error processing paper: {e}")
            continue

Saved HTML content to paper.html
Processing paper: https://arxiv.org/html/2504.06766v1
In try bkock
Error processing paper: tensors used as indices must be long, int, byte or bool tensors
